In [1]:
!pwd

Hand_pose_estimation_3D/arm_and_hand


In [2]:
import os
import sys
import glob

sys.path.append("Hand_pose_estimation_3D/arm_and_hand")
sys.path.append("Hand_pose_estimation_3D")

from dataloader_hand_only_ann import HandLandmarksDataset_ANN

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import os
from datetime import datetime

import open3d as o3d
from utilities import convert_to_left_shoulder_coord
import time

from sklearn.preprocessing import MinMaxScaler

import pandas as pd
from csv_writer import fusion_csv_columns_name
import matplotlib.pyplot as plt

import joblib
import sklearn

from landmarks_scaler import LandmarksScaler

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
arm_hand_fused_names = ["left shoulder", "left elbow", "left hip", "right shoulder",
    "right hip", "WRIST", "THUMB_CMC", "THUMB_MCP", "THUMB_IP", 
    "THUMB_TIP", "INDEX_FINGER_MCP", "INDEX_FINGER_PIP", "INDEX_FINGER_DIP",
    "INDEX_FINGER_TIP", "MIDDLE_FINGER_MCP", "MIDDLE_FINGER_PIP", "MIDDLE_FINGER_DIP",
    "MIDDLE_FINGER_TIP", "RING_FINGER_MCP", "RING_FINGER_PIP", "RING_FINGER_DIP",
    "RING_FINGER_TIP", "PINKY_MCP", "PINKY_PIP", "PINKY_DIP", "PINKY_TIP", "right elbow",
    "RIGHT_WRIST", "RIGHT_THUMB_CMC", "RIGHT_THUMB_MCP", "RIGHT_THUMB_IP", "RIGHT_THUMB_TIP",
    "RIGHT_INDEX_FINGER_MCP", "RIGHT_INDEX_FINGER_PIP", "RIGHT_INDEX_FINGER_DIP",
    "RIGHT_INDEX_FINGER_TIP", "RIGHT_MIDDLE_FINGER_MCP", "RIGHT_MIDDLE_FINGER_PIP",
    "RIGHT_MIDDLE_FINGER_DIP", "RIGHT_MIDDLE_FINGER_TIP", "RIGHT_RING_FINGER_MCP",
    "RIGHT_RING_FINGER_PIP", "RIGHT_RING_FINGER_DIP", "RIGHT_RING_FINGER_TIP",
    "RIGHT_PINKY_MCP", "RIGHT_PINKY_PIP", "RIGHT_PINKY_DIP", "RIGHT_PINKY_TIP"]

In [5]:
x = np.random.rand(10, 3, 21)
w = x[..., 0]
f = x[..., 1:]
f = f.reshape(-1, 3, 5, 4)

In [8]:
f

array([[[[0.3807612 , 0.31952442, 0.6986497 , 0.49749309],
         [0.84796706, 0.54413506, 0.32872708, 0.44849494],
         [0.92631786, 0.02790989, 0.74981266, 0.71885652],
         [0.59314584, 0.03907931, 0.98294464, 0.40656382],
         [0.74721131, 0.24680667, 0.87628607, 0.85082459]],

        [[0.83111214, 0.62421741, 0.45288908, 0.8171305 ],
         [0.69107217, 0.43775876, 0.95818912, 0.17487038],
         [0.00164135, 0.33747378, 0.50858003, 0.68367701],
         [0.31107254, 0.99499431, 0.67788134, 0.36026811],
         [0.78067081, 0.09243998, 0.55274284, 0.29943869]],

        [[0.77121973, 0.54498712, 0.44252888, 0.19807322],
         [0.20133114, 0.79269901, 0.71389982, 0.60368103],
         [0.00724736, 0.70631378, 0.3709822 , 0.6683565 ],
         [0.59318921, 0.40692678, 0.41841214, 0.37461128],
         [0.27794781, 0.67123943, 0.33754146, 0.03224557]]],


       [[[0.11604589, 0.92297099, 0.5199399 , 0.99258598],
         [0.52777819, 0.91189304, 0.68762164, 0.

In [21]:
w

array([[2.84295702e-01, 7.07316202e-01, 8.05505671e-02],
       [2.91299631e-01, 5.73610580e-01, 5.98923876e-01],
       [2.16422675e-01, 4.25619573e-04, 1.36305300e-01],
       [6.23170035e-01, 9.55036953e-01, 9.53546442e-01],
       [7.20842448e-01, 4.77293072e-01, 6.55929641e-01],
       [3.56665844e-01, 2.77237059e-02, 7.82407654e-02],
       [6.32137191e-01, 6.48131652e-01, 4.04640456e-01],
       [5.26875027e-01, 3.24508499e-01, 5.98495756e-01],
       [4.01595455e-01, 7.45276262e-01, 5.46198776e-01],
       [7.88888204e-01, 6.88823492e-01, 1.72792369e-01]])

In [26]:
w = w[..., None]
w = np.repeat(w, 5, axis=-1)

In [32]:
w[..., None]

array([[[[2.84295702e-01],
         [2.84295702e-01],
         [2.84295702e-01],
         [2.84295702e-01],
         [2.84295702e-01]],

        [[7.07316202e-01],
         [7.07316202e-01],
         [7.07316202e-01],
         [7.07316202e-01],
         [7.07316202e-01]],

        [[8.05505671e-02],
         [8.05505671e-02],
         [8.05505671e-02],
         [8.05505671e-02],
         [8.05505671e-02]]],


       [[[2.91299631e-01],
         [2.91299631e-01],
         [2.91299631e-01],
         [2.91299631e-01],
         [2.91299631e-01]],

        [[5.73610580e-01],
         [5.73610580e-01],
         [5.73610580e-01],
         [5.73610580e-01],
         [5.73610580e-01]],

        [[5.98923876e-01],
         [5.98923876e-01],
         [5.98923876e-01],
         [5.98923876e-01],
         [5.98923876e-01]]],


       [[[2.16422675e-01],
         [2.16422675e-01],
         [2.16422675e-01],
         [2.16422675e-01],
         [2.16422675e-01]],

        [[4.25619573e-04],
         [

In [33]:
h = np.concatenate([w[..., None], f], axis=-1)

In [36]:
h[0]

array([[[0.2842957 , 0.3807612 , 0.31952442, 0.6986497 , 0.49749309],
        [0.2842957 , 0.84796706, 0.54413506, 0.32872708, 0.44849494],
        [0.2842957 , 0.92631786, 0.02790989, 0.74981266, 0.71885652],
        [0.2842957 , 0.59314584, 0.03907931, 0.98294464, 0.40656382],
        [0.2842957 , 0.74721131, 0.24680667, 0.87628607, 0.85082459]],

       [[0.7073162 , 0.83111214, 0.62421741, 0.45288908, 0.8171305 ],
        [0.7073162 , 0.69107217, 0.43775876, 0.95818912, 0.17487038],
        [0.7073162 , 0.00164135, 0.33747378, 0.50858003, 0.68367701],
        [0.7073162 , 0.31107254, 0.99499431, 0.67788134, 0.36026811],
        [0.7073162 , 0.78067081, 0.09243998, 0.55274284, 0.29943869]],

       [[0.08055057, 0.77121973, 0.54498712, 0.44252888, 0.19807322],
        [0.08055057, 0.20133114, 0.79269901, 0.71389982, 0.60368103],
        [0.08055057, 0.00724736, 0.70631378, 0.3709822 , 0.6683565 ],
        [0.08055057, 0.59318921, 0.40692678, 0.41841214, 0.37461128],
        [0.08055

In [48]:
h[0]

array([[[0.2842957 , 0.3807612 , 0.31952442, 0.6986497 , 0.49749309],
        [0.2842957 , 0.84796706, 0.54413506, 0.32872708, 0.44849494],
        [0.2842957 , 0.92631786, 0.02790989, 0.74981266, 0.71885652],
        [0.2842957 , 0.59314584, 0.03907931, 0.98294464, 0.40656382],
        [0.2842957 , 0.74721131, 0.24680667, 0.87628607, 0.85082459]],

       [[0.7073162 , 0.83111214, 0.62421741, 0.45288908, 0.8171305 ],
        [0.7073162 , 0.69107217, 0.43775876, 0.95818912, 0.17487038],
        [0.7073162 , 0.00164135, 0.33747378, 0.50858003, 0.68367701],
        [0.7073162 , 0.31107254, 0.99499431, 0.67788134, 0.36026811],
        [0.7073162 , 0.78067081, 0.09243998, 0.55274284, 0.29943869]],

       [[0.08055057, 0.77121973, 0.54498712, 0.44252888, 0.19807322],
        [0.08055057, 0.20133114, 0.79269901, 0.71389982, 0.60368103],
        [0.08055057, 0.00724736, 0.70631378, 0.3709822 , 0.6683565 ],
        [0.08055057, 0.59318921, 0.40692678, 0.41841214, 0.37461128],
        [0.08055

In [59]:
0.54413506 - 0.84796706 - (-3.03832000e-01)

0.0

In [40]:
h.shape

(10, 3, 5, 5)

In [60]:
z = np.zeros_like(h[..., :-1], dtype=np.float32)  # (N, 3, 5, 4)

for i in range(1, h.shape[-1]):
    z[..., i - 1] = h[..., i] - h[..., i - 1]

In [44]:
z

array([[[[ 9.64654982e-02, -6.12367846e-02,  3.79125280e-01,
          -2.01156610e-01],
         [ 5.63671360e-01, -3.03832000e-01, -2.15407984e-01,
           1.19767866e-01],
         [ 6.42022158e-01, -8.98407973e-01,  7.21902769e-01,
          -3.09561342e-02],
         [ 3.08850135e-01, -5.54066529e-01,  9.43865330e-01,
          -5.76380817e-01],
         [ 4.62915603e-01, -5.00404632e-01,  6.29479393e-01,
          -2.54614784e-02]],

        [[ 1.23795938e-01, -2.06894728e-01, -1.71328333e-01,
           3.64241422e-01],
         [-1.62440321e-02, -2.53313408e-01,  5.20430359e-01,
          -7.83318741e-01],
         [-7.05674852e-01,  3.35832430e-01,  1.71106250e-01,
           1.75096982e-01],
         [-3.96243658e-01,  6.83921768e-01, -3.17112976e-01,
          -3.17613229e-01],
         [ 7.33546099e-02, -6.88230829e-01,  4.60302857e-01,
          -2.53304153e-01]],

        [[ 6.90669166e-01, -2.26232616e-01, -1.02458238e-01,
          -2.44455655e-01],
         [ 1.2078

In [64]:
z.shape

(10, 3, 5, 4)

In [65]:
np.sum(z, axis=-1).shape

(10, 3, 5)

In [4]:
DATA_DIR = "data"  
SELECTED_DATE = "2024-*"  # Keep '*' when using glob.glob

train_paths = glob.glob(os.path.join(DATA_DIR, "{}/{}/fine_landmarks_{}_*.csv".format(SELECTED_DATE, SELECTED_DATE, "train")))
val_paths = glob.glob(os.path.join(DATA_DIR, "{}/{}/fine_landmarks_{}_*.csv".format(SELECTED_DATE, SELECTED_DATE, "val")))

body_lines = [[0,2], [0, 3], [2, 4], [3, 4]]
lefthand_lines = [[0, 1], [1, 5], [5, 6], [5, 10], [5, 22], [10, 14], [14, 18], [18, 22], 
    [6, 7], [7, 8], [8, 9], 
    [10, 11], [11, 12], [12, 13], 
    [14, 15], [15, 16], [16, 17], 
    [18, 19], [19, 20], [20, 21], 
    [22, 23], [23, 24], [24, 25]]
train_body_distance_thres = 550
train_leftarm_distance_thres = 550
train_lefthand_distance_thres = 200
val_body_distance_thres = 450
val_leftarm_distance_thres = 450
val_lefthand_distance_thres = 150


input_scaler = MinMaxScaler()
output_scaler = MinMaxScaler()

train_dataset = HandLandmarksDataset_ANN(train_paths, 
    arm_hand_fused_names,
    body_lines, 
    lefthand_lines, 
    train_body_distance_thres, 
    train_leftarm_distance_thres, 
    train_lefthand_distance_thres,
    filter_outlier=True,
    only_keep_frames_contain_lefthand=True,
    cvt_normalized_xy_to_XY=True)

In [5]:
input_scaler.fit_transform(train_dataset._inputs)
output_scaler.fit_transform(train_dataset._outputs)

input_scaler_save_path = os.path.join('.', "input_scaler.pkl")
joblib.dump(input_scaler, input_scaler_save_path)

output_scaler_save_path = os.path.join('.', "output_scaler.pkl")
joblib.dump(output_scaler, output_scaler_save_path)

['./output_scaler.pkl']

In [6]:
input_scaler = LandmarksScaler(scaler_path=input_scaler_save_path)
output_scaler = LandmarksScaler(scaler_path=output_scaler_save_path)

train_dataset = HandLandmarksDataset_ANN(train_paths, 
    arm_hand_fused_names,
    body_lines, 
    lefthand_lines, 
    train_body_distance_thres, 
    train_leftarm_distance_thres, 
    train_lefthand_distance_thres,
    filter_outlier=True,
    only_keep_frames_contain_lefthand=True,
    cvt_normalized_xy_to_XY=True,
    input_scaler=input_scaler,
    output_scaler=output_scaler)
train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=True)

In [7]:
val_dataset = HandLandmarksDataset_ANN(val_paths,
    arm_hand_fused_names,
    body_lines,
    lefthand_lines,
    val_body_distance_thres,
    val_leftarm_distance_thres,
    val_lefthand_distance_thres,
    filter_outlier=True,
    only_keep_frames_contain_lefthand=True,
    cvt_normalized_xy_to_XY=True,
    input_scaler=input_scaler,
    output_scaler=output_scaler)
val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=True)  

In [7]:
arm_hand_fused_names = ["left shoulder", "left elbow", "left hip", "right shoulder", "right hip", 
 "WRIST", "THUMB_CMC", "THUMB_MCP", "THUMB_IP", "THUMB_TIP", "INDEX_FINGER_MCP", 
 "INDEX_FINGER_PIP", "INDEX_FINGER_DIP", "INDEX_FINGER_TIP", "MIDDLE_FINGER_MCP", 
 "MIDDLE_FINGER_PIP", "MIDDLE_FINGER_DIP", "MIDDLE_FINGER_TIP", "RING_FINGER_MCP", 
 "RING_FINGER_PIP", "RING_FINGER_DIP", "RING_FINGER_TIP", "PINKY_MCP", "PINKY_PIP", 
 "PINKY_DIP", "PINKY_TIP", "right elbow"]

In [10]:
body = train_dataset._body_outputs
left_hand = output_scaler.minmax_scaler.inverse_transform(train_dataset._outputs)

In [11]:
body.shape

(14463, 15)

In [12]:
left_hand.shape

(14463, 63)

In [13]:
body = body.reshape(-1, 3, 5)
left_hand = left_hand.reshape(-1, 3, 21)

In [14]:
target = np.concatenate([body, left_hand], axis=-1)

In [15]:
target = np.transpose(target, (0, 2, 1))

In [16]:
target_output = target  # shape: (N, 26, 3)
time_sleep = .00001
x = np.array([[500, 0, 0],
    [0, 0, 0]])
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(x)
lines = [[0, 0]]
colors = [[1, 0, 0] for i in range(len(lines))]
line_set = o3d.geometry.LineSet(
    points=o3d.utility.Vector3dVector(x),
    lines=o3d.utility.Vector2iVector(lines)
)
line_set.colors = o3d.utility.Vector3dVector(colors)

bounding_box = o3d.geometry.LineSet(
    points=o3d.utility.Vector3dVector(np.array([[500,0 ,0], [0, 0, 0]])),
    lines=o3d.utility.Vector2iVector([[0, 0]])
)
    
vis = o3d.visualization.Visualizer()
vis.create_window()
vis.add_geometry(pcd)
vis.add_geometry(line_set)
vis.add_geometry(bounding_box)

for i in range(target_output.shape[0]):
    output = target_output[i, ...] 

    lines = body_lines.copy()
    lines.extend(lefthand_lines)

    points, _ = convert_to_left_shoulder_coord(output,
        arm_hand_fused_names)

    pcd.points = o3d.utility.Vector3dVector(points)
    colors = [[1, 0, 0] for i in range(len(lines))]
    line_set.points = o3d.utility.Vector3dVector(points)  
    line_set.lines = o3d.utility.Vector2iVector(lines) 
    line_set.colors = o3d.utility.Vector3dVector(colors)

    # Draw cuboid
    min_x, min_y, min_z = np.min(points, axis=0)
    max_x, max_y, max_z = np.max(points, axis=0)
    vertices = [
        [min_x, min_y, min_z], [min_x, min_y, max_z], [min_x, max_y, min_z], [min_x, max_y, max_z],
        [max_x, min_y, min_z], [max_x, min_y, max_z], [max_x, max_y, min_z], [max_x, max_y, max_z]
    ]
    vertices = np.array(vertices) 
    edges = [
        [0, 1], [1, 3], [3, 2], [2, 0],  
        [4, 5], [5, 7], [7, 6], [6, 4],  
        [0, 4], [1, 5], [2, 6], [3, 7]   
    ]
    bounding_box.points = o3d.utility.Vector3dVector(vertices)
    bounding_box.lines = o3d.utility.Vector2iVector(edges)
    bbox_colors = [[0, 1, 0] for _ in range(len(edges))]  
    bounding_box.colors = o3d.utility.Vector3dVector(bbox_colors)

    vis.update_geometry(pcd)
    vis.update_geometry(line_set)
    vis.update_geometry(bounding_box)
    vis.poll_events()
    vis.update_renderer()
        
    time.sleep(time_sleep)

vis.destroy_window()

Val set

In [21]:
val_body = val_dataset._body_outputs
val_left_hand = output_scaler.minmax_scaler.inverse_transform(val_dataset._outputs)

In [22]:
val_body = val_body.reshape(-1, 3, 5)
val_left_hand = val_left_hand.reshape(-1, 3, 21)

val_target = np.concatenate([val_body, val_left_hand], axis=-1)
val_target = np.transpose(val_target, (0, 2, 1))

In [23]:
target_output = val_target  # shape: (N, 26, 3)
time_sleep = .001
x = np.array([[500, 0, 0],
    [0, 0, 0]])
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(x)
lines = [[0, 0]]
colors = [[1, 0, 0] for i in range(len(lines))]
line_set = o3d.geometry.LineSet(
    points=o3d.utility.Vector3dVector(x),
    lines=o3d.utility.Vector2iVector(lines)
)
line_set.colors = o3d.utility.Vector3dVector(colors)

bounding_box = o3d.geometry.LineSet(
    points=o3d.utility.Vector3dVector(np.array([[500,0 ,0], [0, 0, 0]])),
    lines=o3d.utility.Vector2iVector([[0, 0]])
)
    
vis = o3d.visualization.Visualizer()
vis.create_window()
vis.add_geometry(pcd)
vis.add_geometry(line_set)
vis.add_geometry(bounding_box)

for i in range(target_output.shape[0]):
    output = target_output[i, ...] 

    lines = body_lines.copy()
    lines.extend(lefthand_lines)

    points, _ = convert_to_left_shoulder_coord(output,
        arm_hand_fused_names)

    pcd.points = o3d.utility.Vector3dVector(points)
    colors = [[1, 0, 0] for i in range(len(lines))]
    line_set.points = o3d.utility.Vector3dVector(points)  
    line_set.lines = o3d.utility.Vector2iVector(lines) 
    line_set.colors = o3d.utility.Vector3dVector(colors)

    # Draw cuboid
    min_x, min_y, min_z = np.min(points, axis=0)
    max_x, max_y, max_z = np.max(points, axis=0)
    vertices = [
        [min_x, min_y, min_z], [min_x, min_y, max_z], [min_x, max_y, min_z], [min_x, max_y, max_z],
        [max_x, min_y, min_z], [max_x, min_y, max_z], [max_x, max_y, min_z], [max_x, max_y, max_z]
    ]
    vertices = np.array(vertices) 
    edges = [
        [0, 1], [1, 3], [3, 2], [2, 0],  
        [4, 5], [5, 7], [7, 6], [6, 4],  
        [0, 4], [1, 5], [2, 6], [3, 7]   
    ]
    bounding_box.points = o3d.utility.Vector3dVector(vertices)
    bounding_box.lines = o3d.utility.Vector2iVector(edges)
    bbox_colors = [[0, 1, 0] for _ in range(len(edges))]  
    bounding_box.colors = o3d.utility.Vector3dVector(bbox_colors)

    vis.update_geometry(pcd)
    vis.update_geometry(line_set)
    vis.update_geometry(bounding_box)
    vis.poll_events()
    vis.update_renderer()
        
    time.sleep(time_sleep)

vis.destroy_window()